In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
os.chdir("./gdrive/My Drive/NLU/Projects/project 1/rok/")

import tensorflow as tf
import numpy as np
import pickle
import warnings

print("tf_version:\t" + tf.__version__)
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"
tf.logging.set_verbosity(tf.logging.ERROR)

from model import Model
from load_embedding import load_embedding
import utils
import tf_utils

logger = utils.Logger("./logs/")
timer = utils.Timer()

!pip install tensorboardcolab
from tensorboardcolab import *
tbc = TensorBoardColab()

Mounted at /content/gdrive/
tf_version:	1.13.1


Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://12fcee36.ngrok.io


In [0]:
# MODEL
tf.app.flags.DEFINE_string("EXPERIMENT", "A", "model selection (A, B, C)")
tf.app.flags.DEFINE_string("MODE", "E", "mode (Experiment - E, Generation - G)")
tf.app.flags.DEFINE_boolean("RESTORE", False, "mode (Experiment - E, Generation - G)")

# -------------------------------------------------------------------------------------------------------------------- #
# DIRECTORIES AND FILES
tf.app.flags.DEFINE_string("DATA_DIR", "./data/", "data directory")
tf.app.flags.DEFINE_string("RESULTS_DIR", "./results/", "results directory")
tf.app.flags.DEFINE_string("MODEL_DIR", "./model/", "saved model directory")
tf.app.flags.DEFINE_string("WORD_EMBEDDINGS_FILE", "wordembeddings-dim100.word2vec", "word embedding file")
tf.app.flags.DEFINE_string("SENTENCES_TRAIN_FILE", "sentences.train", "train file")
tf.app.flags.DEFINE_string("SENTENCES_TEST_FILE", "sentences_test.txt", "test file")
tf.app.flags.DEFINE_string("SENTENCES_EVAL_FILE", "sentences.eval", "evaluation file")
tf.app.flags.DEFINE_string("SENTENCES_CONTINUATION_FILE", "sentences.continuation", "continuation file")

# LANGUAGE MODEL PARAMETERS
tf.app.flags.DEFINE_integer("EMBEDDING_DIM", 100, "word embedding dimension")
tf.app.flags.DEFINE_integer("DOWN_STATE_DIM", 512, "down projection dimension")
tf.app.flags.DEFINE_integer("STATE_DIM", 512, "rnn cell hidden state dimension")
tf.app.flags.DEFINE_integer("VOCABULARY_SIZE", 20000, "vocabulary size")
tf.app.flags.DEFINE_integer("SENT_DIM", 30, "train sentence length")
tf.app.flags.DEFINE_integer("CONT_DIM", 20, "continuation max. sentence length")

# RNN PARAMETERS
tf.app.flags.DEFINE_integer("BATCH_SIZE", 64, "batch size")
tf.app.flags.DEFINE_integer("NUM_EPOCHS", 1, "number of epochs for training")
tf.app.flags.DEFINE_float("LEARNING_RATE", 0.001, "learning rate for rnn")
tf.app.flags.DEFINE_float("MAX_GRAD_NORM", 5.0, "max. norm for gradient clipping")
tf.app.flags.DEFINE_string('f', '', 'tensorflow bug')

FLAGS = tf.app.flags.FLAGS
if FLAGS.EXPERIMENT == "C":
    FLAGS.STATE_DIM = 1024
tf_utils.print_flags(FLAGS)

# -------------------------------------------------------------------------------------------------------------------- #
# LOAD VOCABULARY
with open(FLAGS.RESULTS_DIR + "vocabulary.pkl", "rb") as f:
    vocabulary, word_to_idx, idx_to_word = pickle.load(f)

# -------------------------------------------------------------------------------------------------------------------- #
# RUN
tf.reset_default_graph()
tf.set_random_seed(12345)
np.random.seed(12345)

model = Model(experiment=FLAGS.EXPERIMENT,
              mode=FLAGS.MODE,
              vocabulary_size=FLAGS.VOCABULARY_SIZE,
              embedding_dim=FLAGS.EMBEDDING_DIM,
              state_dim=FLAGS.STATE_DIM,
              down_state_dim=FLAGS.DOWN_STATE_DIM,
              sent_dim=FLAGS.SENT_DIM,
              cont_dim=FLAGS.CONT_DIM,
              initializer=tf.contrib.layers.xavier_initializer(),
              pad_idx=word_to_idx["<pad>"],
              eos_idx=word_to_idx["<eos>"],
              num_epochs=FLAGS.NUM_EPOCHS
              )
tf_utils.trainable_parameters()

saver = tf.train.Saver()
timer.__enter__()

with tf.Session() as session:
    writer = tbc.get_deep_writers("./")
    writer.add_graph(session.graph)
    
    if FLAGS.MODE == "E":
        with tf.name_scope("experiment"):
            if not FLAGS.RESTORE:
                session.run(tf.global_variables_initializer())

                # LOAD EMBEDDING
                if FLAGS.EXPERIMENT == "B":
                    load_embedding(session, word_to_idx, model.embedding_weight,
                                   FLAGS.DATA_DIR + FLAGS.WORD_EMBEDDINGS_FILE, 
                                   FLAGS.EMBEDDING_DIM, FLAGS.VOCABULARY_SIZE)
            else:
                saver.restore(session, FLAGS.MODEL_DIR + "/experiment" + 
                              FLAGS.EXPERIMENT + "/experiment" + 
                              FLAGS.EXPERIMENT + ".ckpt")
                print("Model restored.")
       
            # TRAINING
            summary_op = tf.summary.merge_all()
          
            session.run(model.iterator_op,
                        {model.sentences_file: FLAGS.RESULTS_DIR + "X_train.ids"})

            batch_count = 0
            total_batch = 60000
            while True:
                try:
                    batch_loss, batch_perplexity, _, global_step, summary = session.run([model.loss, model.perplexity, 
                                                                   model.optimize_op, model.global_step, summary_op])
                    writer.add_summary(summary, global_step)
                    epoch = 1
                    if batch_count % 100 == 0:
                        print("epoch: {}/{:<6}batch: {:>5}/{:<10}loss = {:<13.2f}perp = {:<13.2f}".format(epoch, FLAGS.NUM_EPOCHS,
                                                                                                          batch_count + 1,
                                                                                                          total_batch,
                                                                                                          batch_loss,
                                                                                                          batch_perplexity))

                    batch_count += 1
                except tf.errors.OutOfRangeError:
                    break
            
            writer.flush()
            save_path = saver.save(session, FLAGS.MODEL_DIR + "/experiment" + 
                                   FLAGS.EXPERIMENT + "/experiment" + 
                                   FLAGS.EXPERIMENT + ".ckpt")
            print("Model saved in path: %s" % save_path)

            # EVALUATION
            session.run(model.iterator_op, {model.sentences_file: FLAGS.RESULTS_DIR + "X_eval.ids"})
            eval_perplexities = np.array([], dtype=np.float32)
            batch_count = 0
            while True:
                try:
                    batch_perplexities = session.run(model.perplexities)
                    eval_perplexities = np.append(eval_perplexities, batch_perplexities)
                    batch_count += 1
                except tf.errors.OutOfRangeError:
                    break
            print("Evaluation finished.")

            with open(RESULTS_DIR + "groupXX.perplexity" + FLAGS.EXPERIMENT, "w") as f:
                for i in range(eval_perplexities.shape[0]):
                    f.write("%0.3f" % eval_perplexities[i] + "\n")

    elif FLAGS.MODE == "G":
        with tf.name_scope("generation"):
            saver.restore(session, FLAGS.MODEL_DIR + "/experiment" + 
                          FLAGS.EXPERIMENT + "/experiment" + 
                          FLAGS.EXPERIMENT + ".ckpt")
            print("Model restored.")
            
            session.run(model.iterator_op, {model.sentences_file: FLAGS.RESULTS_DIR + "X_cont.ids"})


            continuation_ids = []
            batch_count = 0
            while True:
                try:
                    batch_predictions = session.run(model.predictions)
                    continuation_ids.append(batch_predictions)
                    batch_count = batch_count + 1

                    print(batch_count, end="\r")
                except tf.errors.OutOfRangeError:
                    break

            continuation_ids = np.concatenate(continuation_ids, axis=0)
            print(continuation_ids.shape)

            with open(FLAGS.RESULTS_DIR + "groupXX.continuation", "w") as f:
                for i in range(continuation_ids.shape[0]):
                    try:
                        eos_pos = continuation_ids[i, 1:].tolist().index(int(word_to_idx["<eos>"]))
                    except:
                        eos_pos = 20

                    gen_sent = " ".join([idx_to_word[token_id] if idx < eos_pos else "" for idx, token_id in
                                         enumerate(continuation_ids[i, 1:].tolist())])
                    f.write(gen_sent + "\n")
                
timer.__exit__()
tf_utils.delete_flags(FLAGS)

Command-line Arguments:
EXPERIMENT                         : A
MODE                               : E
RESTORE                            : False
DATA_DIR                           : ./data/
RESULTS_DIR                        : ./results/
MODEL_DIR                          : ./model/
WORD_EMBEDDINGS_FILE               : wordembeddings-dim100.word2vec
SENTENCES_TRAIN_FILE               : sentences.train
SENTENCES_TEST_FILE                : sentences_test.txt
SENTENCES_EVAL_FILE                : sentences.eval
SENTENCES_CONTINUATION_FILE        : sentences.continuation
EMBEDDING_DIM                      : 100
DOWN_STATE_DIM                     : 512
STATE_DIM                          : 512
VOCABULARY_SIZE                    : 20000
SENT_DIM                           : 30
CONT_DIM                           : 20
BATCH_SIZE                         : 64
NUM_EPOCHS                         : 1
LEARNING_RATE                      : 0.001
MAX_GRAD_NORM                      : 5.0



For more inform